# Trabajo final asignatura MD004

## PARTE I 

Imaginemos que trabajas en una empresa de análisis de datos para una compañía de transporte público que opera una flota de autobuses en Barcelona. Debes desarrollar un modelo predictivo para estimar el consumo de combustible de los autobuses en
función de diferentes variables. Dispones de un conjunto de datos que incluye 35 variables explicativas (31 continuas y 4 categóricas).

El dataset que te hacen llegar contiene información recopilada de sensores instalados en los autobuses, así como datos operativos y ambientales. A continuación, se presenta un ejemplo de las variables presentes en el conjunto de datos.

Ejemplo variables continuas:

• Distancia recorrida desde el último repostaje (en kilómetros)

• Velocidad media del autobús durante el trayecto (en kilómetros por hora)

• Carga promedio de pasajeros a bordo

• Temperatura exterior durante el trayecto (en grados Celsius)

• Año de fabricación del autobús

Variables categóricas:

• Tipo de motor del autobús (convencional, eléctrico, híbrido)

• Ruta del autobús (urbana, interurbana)

• Tramo de la semana (fin_de_semana, no_fin_de_semana)

• Condiciones climáticas (soleado, nublado, lluvioso, nevado)

¿Podrías describir que estrategia seguirías para desarrollar un modelo predictivo utilizando este conjunto de datos?

Mi objetivo principal no es solo conseguir un modelo que "acierte", sino un modelo robusto, interpretable (para que la empresa entienda por qué se gasta más o menos) y preparado para producción.

A continuación, detallo la estrategia paso a paso y el porqué de cada decisión:

#### 1. Comprensión del Problema y Definición de la Variable Objetivo
Antes de tocar un solo dato, necesito hablar con negocio. ¿Cómo medimos el consumo? ¿Litros totales por viaje? ¿Litros cada 100 km? Definir matemáticamente la variable objetivo es crucial. Si predecimos "litros por viaje", las rutas más largas siempre tendrán más consumo. Lo ideal sería predecir la eficiencia, ya que normaliza el dato y permite comparar un bus en una ruta corta urbana con otro en una ruta larga interurbana.

#### 2. Análisis Exploratorio de Datos (EDA)
Analizar la distribución estadística de las 35 variables y de la variable objetivo.

Buscaré qué variables continuas tienen una correlación lineal fuerte absoluta (usando el coeficiente de Pearson) o no lineal con el consumo. Observar cómo se distribuye el consumo agrupado por las variables categóricas (ej. boxplot del consumo según tipo de motor o clima). El EDA me permite detectar si hay sensores defectuosos (ej. un autobús registrando 300 km/h o temperaturas de -50ºC en Barcelona) y entender las dinámicas básicas (es obvio que un motor eléctrico consumirá 0 combustible fósil, ¿cómo tratamos esto en el dataset?).

#### 3. Preprocesamiento y Limpieza de Datos
Outliers: Imputaré valores faltantes (usando la media temporal o KNN) o eliminaré registros si el sensor falló catastróficamente. Los valores atípicos reales (un embotellamiento severo) se mantienen; los errores de sensor se descartan.
Encoding: Aplicaré One-Hot Encoding para las 4 variables categóricas (Motor, Ruta, Semana, Clima).

Escalado de Variables Continuas: Aplicaré StandardScaler (media 0, varianza 1) a las 31 variables continuas. El One-Hot Encoding es necesario porque los modelos matemáticos necesitan números, y no hay un orden matemático inherente entre "soleado" y "lluvioso". El escalado es fundamental porque variables en diferentes magnitudes (Distancia en decenas de km vs. Año de fabricación en miles) desestabilizarían algoritmos basados en gradiente descendente (Redes Neuronales) o en distancias (KNN, SVM).

#### 4. Feature Engineering
Crear nuevas columnas a partir de las existentes usando lógica de dominio.

En lugar de "Año de fabricación", calcularé la "Antigüedad del vehículo" (Año actual - Año de fabricación).
Crearé variables de interacción, como "Peso estimado" (Carga promedio de pasajeros $\times$ peso medio por persona).
Podría crear un ratio "Dinámica de conducción" (Velocidad media / Velocidad máxima del trayecto). A menudo, la relación entre los datos originales y el objetivo no es directa. Los modelos no siempre son capaces de deducir que un autobús es viejo solo viendo el número "2012". Dárselo "masticado" como "12 años de antigüedad" facilita que el modelo capte la señal de desgaste del motor.

#### 5. Selección de Modelo
Es un problema de Regresión (predecir un valor numérico continuo). Probaré varios algoritmos en orden de complejidad:

* Regresión Lineal Múltiple (Baseline): Para tener una métrica base. 
* Modelos basados en Árboles (Random Forest, Gradient Boosting como XGBoost): Serán mi apuesta principal. Las relaciones en la física del movimiento rara vez son puramente lineales. El consumo de combustible depende de interacciones complejas (ej. Carga alta de pasajeros + Ruta con pendiente = Consumo exponencialmente mayor). Los ensambles de árboles capturan estas no linealidades e interacciones de forma nativa, son muy potentes con datos tabulares y son robustos frente a outliers.

#### 6. Time-Series Cross Validation o Train/Test Split
Si los datos tienen un componente temporal fuerte (ej. registros a lo largo de los meses), no haré un desorden aleatorio (Random Split). Entrenaré con datos del pasado y validaré con datos del futuro (ej. entrenar con datos de Enero a Octubre, validar en Noviembre y testear en Diciembre). En el mundo real, vamos a predecir el futuro. Si hago una partición aleatoria, podría estar usando datos de un martes soleado aleatorio para predecir el lunes anterior, produciendo Data Leakage (Fuga de información). Evaluar de forma cronológica asegura que el modelo generaliza bien hacia el futuro.

#### 7. Entrenamiento y Optimización de Hiperparámetros
Usaré RandomizedSearchCV o técnicas de optimización bayesiana (como la librería Optuna) para afinar los parámetros de los modelos (profundidad de los árboles, tasa de aprendizaje en XGBoost, etc.). Los hiperparámetros por defecto rara vez extraen el máximo rendimiento de los datos. La optimización bayesiana, vista en el máster, es computacionalmente más eficiente que buscar a fuerza bruta todas las combinaciones posibles (Grid Search).

#### 8. Evaluación con Métricas de Negocio
Mediré el rendimiento usando múltiples métricas:

* RMSE (Root Mean Squared Error): Penará severamente los errores grandes (ej. si el modelo falla por 20 litros en un viaje).
* MAE (Mean Absolute Error): Como ingenieros, le diremos a negocio: "Nuestro modelo se equivoca, en promedio, en ±X litros por viaje". Es la métrica más interpretable para un gerente.
* R²: Para explicar qué porcentaje de la variabilidad del consumo logramos explicar con las 35 variables.

#### 9. Explainable AI
No entregaré el modelo como una simple "caja negra". Utilizaré la librería SHAP (SHapley Additive exPlanations) para analizar cuáles de las 35+ variables dictan el consumo final. Desde una perspectiva de ingeniería y ciencia de datos, si el modelo predice un pico de consumo, la compañía de transportes de Barcelona necesita saber el motivo para tomar decisiones técnicas (ej. "SHAP indica que el aire acondicionado encendido en los autobuses anteriores a 2015 dispara un 18% el consumo, sugerimos renovar esa sub-flota").

Esta sería una arquitectura sólida de principio a fin, contemplando no solo la estadística matemática y algoritmia, sino también la estructura de ingeniería de datos y el valor real para el negocio de transporte.

## PARTE II

Tenemos un dataset que contiene datos cualitativos y cuantitativos de clientes de una empresa de telecomunicaciones india en la que se detallan aspectos de los clientes de la empresa. El objetivo del presente dataset es encontrar accionesconcretas que nos ayuden a prevenir que un cliente haga churn:

• device user’s – device brand (Categorical)

• first_payment_amount – user’s first payment amount(Numeric)

• age – user’s age(Numeric or categorical?)

• city – user’s city(Categorical)

• number_of_cards – #of cards user owns

• payments_initiated – #of payments initiated by user

• payments_failed – #of payments failed

• payments_completed – #of payments completed

• payments_completed_amount_first_7days – amt of payment completed in first 7 days of joining

• reward_purchase_count_first_7days – #of rewards claimed in first 7 days

• coins_redeemed_first_7days – coins redeemed in first 7 days

• is_referral – is user a referred user

• visits_feature_1 – #of visits made by user to product feature 1

• visits_feature_2 – #of visits made by user to product feature 2

• given_permission_1 – has user given permission 1

• given_permission_2 – has user given permission 2

• user_id – user identifier

• is_churned – whether user churned

In [1]:
library(readr)
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
datos_churn <- read_csv("MD004_ACFinal_customer_churn_data.csv")

Rows: 104143 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): device, city
dbl (15): first_payment_amount, age, number_of_cards, payments_initiated, pa...
lgl  (1): is_referral

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
head(datos_churn)

device,first_payment_amount,age,city,number_of_cards,payments_initiated,payments_failed,payments_completed,payments_completed_amount_first_7days,reward_purchase_count_first_7days,coins_redeemed_first_7days,is_referral,visits_feature_1,visits_feature_2,given_permission_1,given_permission_2,user_id,is_churned
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
samsung,0,20,Ahmedabad,2,1,0,1,0,NA,0,FALSE,3,0,1,0,269438,0
xiaomi,0,20,Surat,1,1,0,1,0,NA,0,TRUE,0,0,1,0,139521,0
xiaomi,0,20,Kullu,1,2,1,1,0,0,0,TRUE,0,0,0,1,307352,0
oneplus,0,20,Mumbai,2,4,1,2,322,2,20,FALSE,0,0,1,1,456424,0
apple,0,20,Pune,2,1,0,1,0,0,0,FALSE,0,1,1,1,398779,0
oppo,0,20,Ahmedabad,0,1,0,1,0,0,0,TRUE,0,0,1,1,136656,0


In [4]:
str(datos_churn)

spc_tbl_ [104,143 × 18] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ device                               : chr [1:104143] "samsung" "xiaomi" "xiaomi" "oneplus" ...
 $ first_payment_amount                 : num [1:104143] 0 0 0 0 0 0 0 0 0 0 ...
 $ age                                  : num [1:104143] 20 20 20 20 20 20 20 20 20 20 ...
 $ city                                 : chr [1:104143] "Ahmedabad" "Surat" "Kullu" "Mumbai" ...
 $ number_of_cards                      : num [1:104143] 2 1 1 2 2 0 0 0 0 0 ...
 $ payments_initiated                   : num [1:104143] 1 1 2 4 1 1 1 17 1 1 ...
 $ payments_failed                      : num [1:104143] 0 0 1 1 0 0 0 0 0 0 ...
 $ payments_completed                   : num [1:104143] 1 1 1 2 1 1 1 12 1 1 ...
 $ payments_completed_amount_first_7days: num [1:104143] 0 0 0 322 0 0 0 143 0 0 ...
 $ reward_purchase_count_first_7days    : num [1:104143] NA NA 0 2 0 0 0 6 0 NA ...
 $ coins_redeemed_first_7days           : num [1:104143] 0 0 0 20 0 0 0 

In [5]:
summary(datos_churn)

    device          first_payment_amount      age            city          
 Length:104143      Min.   :   0.00      Min.   :20.00   Length:104143     
 Class :character   1st Qu.:   2.00      1st Qu.:27.00   Class :character  
 Mode  :character   Median :  12.00      Median :31.00   Mode  :character  
                    Mean   :  34.77      Mean   :32.69                     
                    3rd Qu.:  37.00      3rd Qu.:36.00                     
                    Max.   :4370.00      Max.   :80.00                     
                                         NA's   :142                       
 number_of_cards  payments_initiated payments_failed   payments_completed
 Min.   : 0.000   Min.   :  1.000    Min.   : 0.0000   Min.   :  1.000   
 1st Qu.: 1.000   1st Qu.:  1.000    1st Qu.: 0.0000   1st Qu.:  1.000   
 Median : 1.000   Median :  2.000    Median : 0.0000   Median :  1.000   
 Mean   : 1.989   Mean   :  2.847    Mean   : 0.4399   Mean   :  1.831   
 3rd Qu.: 3.000   3rd 

Al observar esta primera carga de los datos, hemos identificado varios puntos críticos que debemos corregir antes de lanzarnos al análisis exploratorio (EDA). Aquí tienes nuestro diagnóstico y la hoja de ruta que seguiremos:

#### 1. ¿Qué vemos en esta primera carga?
* Variables como device y city aparecen como texto (chr), y variables binarias como is_churned, given_permission_1 y given_permission_2 aparecen como numéricas. Para que R las procese correctamente en modelos y gráficas, deben ser factores.
* Problema serio de valores faltantes (NAs): 
* * reward_purchase_count_first_7days: Tiene 23,264 NAs (casi un 22% del dataset). Es una cifra muy alta.
* * Cluster de NAs: Hay un grupo recurrente de 472 NAs en varias columnas de pagos y tarjetas. Esto sugiere que hay usuarios que quizás no completaron su registro o el sistema no capturó sus datos de actividad inicial.
* * visits_feature_1/2: Tienen 2,646 NAs.
* Variables de identificación: user_id es un número, pero para nosotros es solo una etiqueta. No debe participar en cálculos estadísticos.
* Outliers potenciales: En first_payment_amount, el máximo es 4370 pero la mediana es solo 12. Habrá que ver si esos valores extremos son errores o clientes "ballena".

#### 2. Pasos para corregir el dataset
Para dejar el dataset listo, vamos a ejecutar estos pasos de limpieza:

Paso A: Ajuste de Tipos de Datos
* Convertiremos las variables categóricas a factores y eliminaremos el user_id del análisis (pero guardándolo si fuera necesario identificar casos luego).

Paso B: Gestión de NAs
* Para los 472 NAs recurrentes: Dado que representan menos del 0.5% de los 104,143 registros, lo más limpio es eliminarlos.
* Para las recompensas y visitas: Debemos decidir si un NA significa "cero". Viendo que el mínimo en esas columnas es 0, es muy probable que el sistema registre NA cuando no ha habido actividad. Los transformaremos en 0.
* Para la edad: Imputaremos la mediana para no perder esos 142 registros.

In [6]:
df_clean <- datos_churn

In [7]:
df_clean <- df_clean %>% 
  filter(!is.na(payments_initiated))

In [8]:
df_clean <- df_clean %>% 
  mutate(
    reward_purchase_count_first_7days = replace_na(reward_purchase_count_first_7days, 0),
    visits_feature_1 = replace_na(visits_feature_1, 0),
    visits_feature_2 = replace_na(visits_feature_2, 0)
  )

In [9]:
df_clean$age[is.na(df_clean$age)] <- median(df_clean$age, na.rm = TRUE)

In [10]:
cols_categoricas <- c("device", "city", "is_referral", 
                      "given_permission_1", "given_permission_2", "is_churned")

In [11]:
df_clean[cols_categoricas] <- lapply(df_clean[cols_categoricas], as.factor)

In [13]:
n_duplicados <- sum(duplicated(df_clean))
print(paste("Número de filas duplicadas encontradas:", n_duplicados))

[1] "Número de filas duplicadas encontradas: 0"


In [14]:
df_clean <- df_clean %>% select(-user_id)

In [15]:
summary(df_clean)

     device      first_payment_amount      age               city      
 xiaomi :24895   Min.   :   0.00      Min.   :20.00   NCR      :14561  
 samsung:16582   1st Qu.:   2.00      1st Qu.:27.00   Bangalore:13641  
 apple  :15743   Median :  12.00      Median :31.00   Mumbai   :10432  
 oneplus:14547   Mean   :  34.76      Mean   :32.69   Hyderabad: 9699  
 vivo   : 8488   3rd Qu.:  37.00      3rd Qu.:36.00   Pune     : 4463  
 (Other):23298   Max.   :4370.00      Max.   :80.00   (Other)  :45054  
 NA's   :  118                                        NA's     : 5821  
 number_of_cards  payments_initiated payments_failed   payments_completed
 Min.   : 0.000   Min.   :  1.000    Min.   : 0.0000   Min.   :  1.000   
 1st Qu.: 1.000   1st Qu.:  1.000    1st Qu.: 0.0000   1st Qu.:  1.000   
 Median : 1.000   Median :  2.000    Median : 0.0000   Median :  1.000   
 Mean   : 1.989   Mean   :  2.847    Mean   : 0.4399   Mean   :  1.831   
 3rd Qu.: 3.000   3rd Qu.:  3.000    3rd Qu.: 0.0000  

In [16]:
df_clean <- df_clean %>%
  mutate(
    city = as.character(city), # Lo pasamos a texto temporalmente para editarlo
    city = replace_na(city, "Unknown"),
    city = as.factor(city),    # Lo volvemos a hacer factor
    
    device = as.character(device),
    device = replace_na(device, "Unknown"),
    device = as.factor(device)
  )

Al observar este nuevo summary, podemos confirmar que:

1. Las variables clave ya son factores: is_churned, given_permission, is_referral, etc., ya están categorizadas (vemos los conteos de 0 y 1, o TRUE/FALSE).
2. Limpieza de actividad completada: Ya no vemos los 23,000 NAs en recompensas ni en visitas. Ahora tenemos valores reales con los que trabajar.
3. Variable objetivo: Vemos que tenemos 29,774 casos de churn (1) frente a 73,897 de no churn (0). Es una proporción de casi el 29%, lo cual es fantástico para entrenar un modelo más adelante.

Sin embargo, para que el dataset esté perfecto antes de empezar el EDA, todavía nos quedan dos pequeñas "espinas" que debemos decidir cómo tratar:
* city (5,821 NAs): Tenemos 5,821 clientes sin ciudad. Representan un 5.5% del total. Como es una variable categórica, no podemos usar la mediana.
* device (118 NAs): Son pocos, pero ahí están.

Para no perder esos 5,800 datos (que pueden ser valiosos en las otras columnas), no los borraremos. En su lugar, vamos a etiquetarlos como "Unknown". Así, nuestro EDA podrá decirnos si, por ejemplo, los usuarios que no informan su ciudad tienen más tendencia a irse o no.

In [17]:
sum(is.na(df_clean))

[1] 0

In [19]:
n_duplicados <- sum(duplicated(df_clean))
print(paste("Número de filas duplicadas encontradas:", n_duplicados))

[1] "Número de filas duplicadas encontradas: 825"


In [20]:
df_clean <- df_clean %>% distinct()

In [22]:
n_duplicados <- sum(duplicated(df_clean))
print(paste("Número de filas duplicadas encontradas:", n_duplicados))

[1] "Número de filas duplicadas encontradas: 0"


In [21]:
print(paste("Registros finales:", nrow(df_clean)))

[1] "Registros finales: 102846"


In [23]:
summary(df_clean$is_churned)

0     1 
73772 29074